# Sequence generation for the pseudoRWM set of experiments

## Import statements and utilities

In [1]:
import numpy as np
import os
import matlab.engine
from alive_progress import alive_bar
import pandas as pd

In [2]:
def shuffle_along_axis(arr, axis):
    idx = np.random.rand(*arr.shape).argsort(axis=axis)
    return np.take_along_axis(arr,idx,axis=axis)

def shuffled(arr):
    arr_shuffled = arr.copy()
    np.random.shuffle(arr_shuffled)
    return(arr_shuffled)

def choose_n_and_delete(arr, N):
    chosen = np.random.choice(arr, size=N, replace=False)
    arr = np.delete(arr, np.where(np.isin(arr, chosen)))
    return chosen, arr

## Settings

In [3]:
exp_type = "pseudoRWMReps"  # choose from ["pseudoRWM", "pseudoRWMCtrl", "pseudoRWMConf", "pseudoRWMReps"]
num_conditions = 10 # number of different file sequences to generate (change if needed)
reps_dict = {"pseudoRWM": 12, "pseudoRWMCtrl": 12, "pseudoRWMConf": 11, "pseudoRWMReps": 11}  # number of repetitions after the first presentation
num_reps = reps_dict[exp_type] # number of stimulus repetitions (after first presentation; change if needed)
exact_reps =  True if exp_type in ["pseudoRWMConf", "pseudoRWMReps"] else False  # should the number of repetitions be exact or ok to exceed by 1?
use_matlab = True  # use matlab to call createstimsequence?

## Start MATLAB if needed

In [4]:
if use_matlab:
    m = matlab.engine.start_matlab()
    m.addpath(m.genpath("C:/Gaia/CCN Lab/Utilities/sequences/pseudoRWM/"), nargout=0)
else:
    from numpy.matlib import repmat

## Main sequence creation function

In [46]:
class pseudoRWMSequenceMaker:

    def __init__(self, exp_type, num_reps, use_matlab=True, num_conditions=10, exact_reps=False):

        # ================ SETTINGS ==========================================================================
        assert exp_type in ["pseudoRWM", "pseudoRWMCtrl", "pseudoRWMConf", "pseudoRWMReps"], f"{exp_type} is not a valid exp_type"
        self.exp_type = exp_type
        self.num_reps = num_reps
        self.num_conditions = num_conditions
        self.use_matlab = use_matlab
        self.exact_reps = exact_reps
        self.to_dir = f"C:/Gaia/CCN Lab/Utilities/sequences/pseudoRWM/{exp_type}/"
        self.has_group_row = self.exp_type in ["pseudoRWMConf", "pseudoRWMReps"]

        self.num_keys = 3
        self.max_stims = 6
        self.block_structure = np.array([6, 6, 6, 6, 6, 6]) # 6 blocks: 3*type (Points vs Goals)
        self.num_blocks = len(self.block_structure)
        self.trial_type_structure = np.arange(self.num_blocks)%2
        self.num_goals_blocks = self.num_blocks - np.sum(self.trial_type_structure)
        self.num_goals_trials = np.sum(self.block_structure[np.where(self.trial_type_structure==0)])*(self.num_reps+1)

        # R: columns represent keyboard keys
        # each column says how many stimuli will be associated with that key
        # e.g., 1, 2, 3 means 1 stimulus will be associated with key 0, 2, with key 1, 3 with key 2, 
        # for a total of 6 items
        if self.exp_type in ["pseudoRWM", "pseudoRWMCtrl"]:
            self.R = np.vstack((
                np.array([1, 2, 3]),
                np.array([2, 2, 2]), 
                np.array([1, 2, 3]),
                np.array([2, 2, 2]), 
                np.array([1, 2, 3]),
                np.array([2, 2, 2])
            ))
        else:
            self.R = np.vstack((
                np.array([2, 2, 2]), 
                np.array([2, 2, 2]), 
                np.array([2, 2, 2]), 
                np.array([2, 2, 2]), 
                np.array([2, 2, 2]), 
                np.array([2, 2, 2])
            ))

        # goal images  
        self.all_fracts = np.arange(1, (self.num_goals_trials*2)+1)  # all fractal image numbers 
        self.all_ctrl_goal_sets = [[5, 17], [25, 26], [24, 30], [56, 59], [76, 77]]*2 # fractals to use in the Ctrl version includes goal and non-goal


    def _get_grouped_goal_sequences(self, all_goal_images, all_nongoal_images, trial_types, blocks, block_groups):
        return


    def make_sequences(self):
        with alive_bar(self.num_conditions, title=f"Making sequences", force_tty=True) as bar:
            for s_i in range(self.num_conditions):
                # ================ BLOCKS ===========================================================================
                # blocks: array of length n with each element representing the block's set size
                blocks = self.block_structure.copy()

                # ================ TRIAL TYPES =======================================================================
                # trial_types: array  where each element represent the sequence
                # of trial types (1 = Points, 0 = Goals) a participant will experience
                trial_types = self.trial_type_structure.copy()
                for i in range(0, self.num_blocks//2, 2):
                    np.random.shuffle(trial_types[i:i+2]) 

                # ================ RULES =============================================================================
                # block_rules: a list where each element is a dictionary, with keys representing a stimulus and
                # values representing the solution for each stimulus
                # here we shuffle within rows (only the first three columns), so that it's not always the same keys 
                # that have 1, 2, or 3 stimuli associated with them (no difference for rows with 2, 2, 2) 
                R_i = shuffle_along_axis(self.R, axis=1)  # mix up stim/action within the rule
                block_rules = []
                for block_i in range(self.num_blocks):
                    block_rules.append({i: k for i, k in enumerate([key_i for key_i in range(self.num_keys) for _ in range(R_i[block_i, key_i])])})

                # ================ STIMULI ===========================================================================
                # stim_sets: stimulus sets (folders from where images will be taken for each block)
                stim_sets = (np.random.permutation(self.num_blocks)+1)
        
                # block_stimuli: will contain dictionaries with an image number for each stimulus
                block_stimuli = []
                for block_i, ns in enumerate(blocks):
                    block_stimuli.append({i: s for i, s in enumerate((np.random.permutation(self.max_stims)+1)[0:ns])})

                # block_seqprototypes: will contain dictionaries for each participant, with keys representing a set size and
                # values as lists with a sequence of stimuli to be presented
                # create a prototype (corresponding to stimuli rather than stimulus images) for each set size
                block_seqprototypes = []
                
                # block_sequences: maps block_seqprototypes to corresponding stimulus sequences based on block_stimuli
                block_sequences = []
                
                for block_i, ns in enumerate(blocks):
                    criterion_passed = False
                    if self.use_matlab:
                        while not criterion_passed:
                            temp_seqprototype = np.squeeze(np.array(m.createstimsequence(m.double(int(self.num_reps)), m.double(int(ns))))).astype(int)
                            # if we want the exact number of repetitions across stimuli, we need to pass this criterion before accepting the sequence
                            criterion_passed = np.all(np.unique(temp_seqprototype, return_counts=True)[1]==(self.num_reps+1)) or (not self.exact_reps)
                            # print(s_i, block_i, criterion_passed, np.unique(temp_seqprototype, return_counts=True)[1])
                        block_seqprototypes.append(temp_seqprototype)
                    else:
                        # worse (but faster) alternative if createstimsequence doesn't work
                        temp_seqprototype = []
                        for _ in range(self.num_reps+1):
                            temp_seqprototype = np.hstack((temp_seqprototype, (shuffled(np.arange(1, ns+1)))))
                        block_seqprototypes.append(temp_seqprototype)
                    # turn into stimuli (stimulus image number)
                    block_sequences.append(np.vectorize((block_stimuli[block_i]).get)(block_seqprototypes[block_i]-1))

                # ================ GROUPS ===========================================================================
                # only for some experiments, we divide stimuli into three groups that will receive different treatments 
                # in terms of goal image presentation
                # we need to ensure that stimuli in the same group have different correct keys
                # because for these experiments each key is associated with two consecutive stimulus numbers, we can do
                # this by making sure stimuli of the same group are associated with stimuli that are one number apart
                # we also need to randomize which group is associated with the first, second, or third set of solutions
                # block_groups: will contain dictionaries with a group number for each stimulus
                block_groups = []
                for block_i, ns in enumerate(blocks):
                    block_groups.append({i: g for i, g in enumerate(np.tile(np.random.permutation(ns//2), 2))})

                # ================ GOAL IMAGES ======================================================================
                all_images = shuffled(self.all_fracts)
                if self.exp_type == "pseudoRWM":
                    goal_images = all_images[0:len(all_images)//2] 
                    nongoal_images = all_images[len(all_images)//2:]

                elif self.exp_type == "pseudoRWMCtrl":
                    goal_set = self.all_ctrl_goal_sets[s_i]
                    # counterbalance which image is the goal and which one is the non-goal across participants
                    goal_images = np.repeat(goal_set[int(s_i+1 >= self.num_conditions//2)], self.num_goals_trials)
                    nongoal_images = np.repeat(goal_set[1-int(s_i+1 >= self.num_conditions//2)], self.num_goals_trials)

                elif self.exp_type == "pseudoRWMConf":
                    all_goal_images = all_images[0:len(all_images)//2] 
                    all_nongoal_images = all_images[len(all_images)//2:]
                    goal_images = np.array([])
                    nongoal_images = np.array([])
                    # for each block, we need 2 + 2 + 4 sets of goal/nongoal image pairs
                    group0_goal_images, all_goal_images = choose_n_and_delete(all_goal_images, (self.num_goals_blocks, 2))
                    group1_goal_images, all_goal_images = choose_n_and_delete(all_goal_images, (self.num_goals_blocks, 2))
                    group2_goal_images, all_goal_images = choose_n_and_delete(all_goal_images, (self.num_goals_blocks, 4))

                    group0_nongoal_images, all_nongoal_images = choose_n_and_delete(all_nongoal_images, (self.num_goals_blocks, 2))
                    group1_nongoal_images, all_nongoal_images = choose_n_and_delete(all_nongoal_images, (self.num_goals_blocks, 2))
                    group2_nongoal_images, all_nongoal_images = choose_n_and_delete(all_nongoal_images, (self.num_goals_blocks, 4))
                    # block_goal_sequences will contain a dictionary for each block where keys are stimuli IDs and values are sequences
                    # of goal images for that stimulus
                    # for block_i in range(self.num_goals_blocks):
                    for goal_block_i, block_i in enumerate(np.where(trial_types==0)[0]):
                        ns = blocks[block_i]
                        # dictionary for this block where keys are stimuli IDs and values are sequences of goal images for that stimulus
                        block_goal_sequence_i = {}
                        block_goal_sequence_i = {}
                        # dictionary for this block where keys are stimuli IDs and values are sequences of nongoal images for that stimulus
                        block_nongoal_sequence_i = {}
                        block_nongoal_sequence_i = {}
                        
                        # form goal and nongoal sequences for each group 
                        group0_goal_sequences = np.repeat(group0_goal_images[goal_block_i], (self.num_reps+1)//2)
                        group1_goal_sequences = np.hstack((
                            np.repeat(group1_goal_images[goal_block_i], (self.num_reps+1)//4), 
                            np.repeat(group1_nongoal_images[goal_block_i], (self.num_reps+1)//4)))
                        group2_goal_sequences = np.repeat(group2_goal_images[goal_block_i], (self.num_reps+1)//4)
                        
                        group0_nongoal_sequences = np.repeat(group0_nongoal_images[goal_block_i], (self.num_reps+1)//2)
                        group1_nongoal_sequences = np.hstack((
                            np.repeat(group1_nongoal_images[goal_block_i], (self.num_reps+1)//4), 
                            np.repeat(group1_goal_images[goal_block_i], (self.num_reps+1)//4)))
                        group2_nongoal_sequences = np.repeat(group2_nongoal_images[goal_block_i], (self.num_reps+1)//4)

                        # shuffle goal and nongoal images consistently for the same stimulus and separately for stimuli of the same group
                        first_stim_order = np.random.permutation(self.num_reps+1)
                        second_stim_order = np.random.permutation(self.num_reps+1)
                        
                        # get which stimulus is associated with which group 
                        group_to_stim_dict = {}
                        for key, value in block_groups[block_i].items():
                            if value not in group_to_stim_dict:
                                group_to_stim_dict[value] = []
                            group_to_stim_dict[value].append(key)
                        # group_to_stim_dict[0][0] = the first stimulus of the first group, 
                        # group_to_stim_dict[0][1] = the second stimulus of the first group, etc.
                        # create a goal and a nongoal sequence for each block and stimulus
                        for g_i, (group_goal_sequence, group_nongoal_sequence) in enumerate(zip(
                                [group0_goal_sequences, group1_goal_sequences, group2_goal_sequences], 
                                [group0_nongoal_sequences, group1_nongoal_sequences, group2_nongoal_sequences])):
                            for stim_n, stim_order in enumerate([first_stim_order, second_stim_order]):
                                # goal sequences
                                block_goal_sequence_i[group_to_stim_dict[g_i][stim_n]] = group_goal_sequence[stim_order]
                                # nongoal_sequences
                                block_nongoal_sequence_i[group_to_stim_dict[g_i][stim_n]] = group_nongoal_sequence[stim_order]
                       
                        # create the full goal/nongoal sequences based on where stimuli are located
                        tempgoalseq = np.empty((self.num_reps+1)*ns)
                        tempnongoalseq = np.empty((self.num_reps+1)*ns)
                        for stim_i in range(ns):
                            tempgoalseq[(block_seqprototypes[block_i]-1)==stim_i] = block_goal_sequence_i[stim_i]
                            tempnongoalseq[(block_seqprototypes[block_i]-1)==stim_i] = block_nongoal_sequence_i[stim_i]
                        goal_images = np.hstack((goal_images, tempgoalseq))
                        nongoal_images = np.hstack((nongoal_images, tempnongoalseq))

                elif self.exp_type == "pseudoRWMReps":
                    all_goal_images = all_images[0:len(all_images)//2] 
                    all_nongoal_images = all_images[len(all_images)//2:]
                    goal_images = np.array([])
                    nongoal_images = np.array([])
                    # for each block, we need 2 + 2 + 4 sets of goal/nongoal image pairs
                    group0_goal_images, all_goal_images = choose_n_and_delete(all_goal_images, (self.num_goals_blocks, 6))
                    group1_goal_images, all_goal_images = choose_n_and_delete(all_goal_images, (self.num_goals_blocks, 10))
                    group2_goal_images, all_goal_images = choose_n_and_delete(all_goal_images, (self.num_goals_blocks, 8))

                    group0_nongoal_images, all_nongoal_images = choose_n_and_delete(all_nongoal_images, (self.num_goals_blocks, 6))
                    group1_nongoal_images, all_nongoal_images = choose_n_and_delete(all_nongoal_images, (self.num_goals_blocks, 10))
                    group2_nongoal_images, all_nongoal_images = choose_n_and_delete(all_nongoal_images, (self.num_goals_blocks, 8))
                    # block_goal_sequences will contain a dictionary for each block where keys are stimuli IDs and values are sequences
                    # of goal images for that stimulus
                    # for block_i in range(self.num_goals_blocks):
                    for goal_block_i, block_i in enumerate(np.where(trial_types==0)[0]):
                        ns = blocks[block_i]
                        # dictionary for this block where keys are stimuli IDs and values are sequences of goal images for that stimulus
                        block_goal_sequence_i = {}
                        block_goal_sequence_i = {}
                        # dictionary for this block where keys are stimuli IDs and values are sequences of nongoal images for that stimulus
                        block_nongoal_sequence_i = {}
                        block_nongoal_sequence_i = {}
                        
                        # form goal and nongoal sequences for each group 
                        one_fourth = (self.num_reps+1)//4  # 3 if there are 11(+1) reps
                        one_sixth = (self.num_reps+1)//6   # 2 if there are 11(+1) reps
                        
                        group0_goal_sequences = np.hstack((
                            np.repeat(group0_goal_images[goal_block_i][0:one_fourth], one_fourth),
                            group0_goal_images[goal_block_i][one_fourth:]))
                        group1_goal_sequences = np.hstack((
                            np.repeat(group1_goal_images[goal_block_i][0], one_fourth),
                            group1_goal_images[goal_block_i][1:]))
                        group2_goal_sequences = np.hstack((
                            np.repeat(group2_goal_images[goal_block_i][0:one_sixth], one_fourth),
                            group2_goal_images[goal_block_i][one_sixth:]))
                        
                        group0_nongoal_sequences = np.hstack((
                            np.repeat(group0_nongoal_images[goal_block_i][0:one_fourth], one_fourth),
                            group0_nongoal_images[goal_block_i][one_fourth:]))
                        group1_nongoal_sequences = np.hstack((
                            np.repeat(group1_nongoal_images[goal_block_i][0], one_fourth),
                            group1_nongoal_images[goal_block_i][1:]))
                        group2_nongoal_sequences = np.hstack((
                            np.repeat(group2_nongoal_images[goal_block_i][0:2], one_fourth),
                            group2_nongoal_images[goal_block_i][2:]))

                        # shuffle goal and nongoal images consistently for the same stimulus and separately for stimuli of the same group
                        first_stim_order = np.random.permutation(self.num_reps+1)
                        second_stim_order = np.random.permutation(self.num_reps+1)
                        
                        # get which stimulus is associated with which group 
                        group_to_stim_dict = {}
                        for key, value in block_groups[block_i].items():
                            if value not in group_to_stim_dict:
                                group_to_stim_dict[value] = []
                            group_to_stim_dict[value].append(key)
                        # group_to_stim_dict[0][0] = the first stimulus of the first group, 
                        # group_to_stim_dict[0][1] = the second stimulus of the first group, etc.
                        # create a goal and a nongoal sequence for each block and stimulus
                        for g_i, (group_goal_sequence, group_nongoal_sequence) in enumerate(zip(
                                [group0_goal_sequences, group1_goal_sequences, group2_goal_sequences], 
                                [group0_nongoal_sequences, group1_nongoal_sequences, group2_nongoal_sequences])):
                            for stim_n, stim_order in enumerate([first_stim_order, second_stim_order]):
                                # goal sequences
                                block_goal_sequence_i[group_to_stim_dict[g_i][stim_n]] = group_goal_sequence[stim_order]
                                # nongoal_sequences
                                block_nongoal_sequence_i[group_to_stim_dict[g_i][stim_n]] = group_nongoal_sequence[stim_order]
                       
                        # create the full goal/nongoal sequences based on where stimuli are located
                        tempgoalseq = np.empty((self.num_reps+1)*ns)
                        tempnongoalseq = np.empty((self.num_reps+1)*ns)
                        for stim_i in range(ns):
                            tempgoalseq[(block_seqprototypes[block_i]-1)==stim_i] = block_goal_sequence_i[stim_i]
                            tempnongoalseq[(block_seqprototypes[block_i]-1)==stim_i] = block_nongoal_sequence_i[stim_i]

                        goal_images = np.hstack((goal_images, tempgoalseq))
                        nongoal_images = np.hstack((nongoal_images, tempnongoalseq))

                # ================ CSV FILE ==========================================================================
                # create csv
                # rows: stim, correct key, set size, blocks, img_folders, img_nums, trial_type, goal_img, nongoal_img
                goal_stim_count = 0

                for block_i, ns in enumerate(blocks):
                    block_length = (self.num_reps + 1) * ns  # number of trials in a block
                    
                    this_block = np.full((9+self.has_group_row, block_length), np.nan)

                    _, unique_idx = np.unique(block_sequences[block_i], return_index=True)
                    block_cond = trial_types[block_i]
                    
                    this_block[0] = block_seqprototypes[block_i]  # stimulus number
                    this_block[1] = np.vectorize((block_rules[block_i]).get)(block_seqprototypes[block_i]-1) # correct key for the stimulus ID
                    this_block[2] = np.repeat(ns, block_length)  # set size
                    this_block[3] = np.repeat(block_i+1, block_length)  # block number
                    this_block[4] = np.repeat(stim_sets[block_i], block_length)  # image folder
                    this_block[5] = block_sequences[block_i]  # stimulus number
                    this_block[6] = np.repeat(block_cond, block_length)  # trial type

                    if block_cond == 0:
                            this_block[7] = goal_images[goal_stim_count:goal_stim_count+block_length]
                            this_block[8] = nongoal_images[goal_stim_count:goal_stim_count+block_length]
                            goal_stim_count += block_length
                    
                    if self.has_group_row:
                        this_block[9] = np.vectorize((block_groups[block_i]).get)(block_seqprototypes[block_i]-1) # group for the stimulus ID

                    if block_i == 0:
                        train_seq = this_block
                        unique_stims = this_block[:, unique_idx]
                    else:
                        train_seq = np.column_stack((train_seq, this_block))
                        unique_stims = np.column_stack((unique_stims, this_block[:, unique_idx]))

                # save output
                np.savetxt(f"{self.to_dir}seq{s_i+1}_learning.csv", train_seq, delimiter=",")
                bar()

        return 

## Create sequences

In [47]:
# seqmkr = pseudoRWMSequenceMaker(exp_type=exp_type, num_reps=num_reps, num_conditions=num_conditions, use_matlab=use_matlab, exact_reps=exact_reps)
seqmkr = pseudoRWMSequenceMaker(exp_type=exp_type, num_reps=num_reps, num_conditions=1, use_matlab=False, exact_reps=exact_reps)
seqmkr.make_sequences()

Making sequences |████████████████████████████████████████| 1/1 [100%] in 0.1s (8.62/s)                                 


## Checks

In [48]:
exp_type = "pseudoRWMReps"

df = pd.read_csv(f"{os.path.dirname(os.getcwd())}/{exp_type}/seq1_learning.csv", header=None).T

colnames = ["stim", "correct_key", "set_size", "block", "img_folder", "img_num", "trial_type", "goal_img", "nongoal_img"]
if exp_type in ["pseudoRWMConf", "pseudoRWMReps"]:
    colnames += ["group"]
df.columns = colnames

In [49]:
df.groupby(["block", "group"]).goal_img.unique()

block  group
1.0    0.0              [239.0, 20.0, 284.0, 306.0, 392.0, 149.0]
       1.0      [418.0, 294.0, 95.0, 90.0, 383.0, 412.0, 367.0...
       2.0      [318.0, 134.0, 240.0, 25.0, 370.0, 241.0, 400....
2.0    0.0                                                  [nan]
       1.0                                                  [nan]
       2.0                                                  [nan]
3.0    0.0                                                  [nan]
       1.0                                                  [nan]
       2.0                                                  [nan]
4.0    0.0             [143.0, 231.0, 335.0, 102.0, 349.0, 250.0]
       1.0      [362.0, 388.0, 401.0, 281.0, 60.0, 330.0, 377....
       2.0      [234.0, 414.0, 373.0, 135.0, 228.0, 263.0, 415...
5.0    0.0               [365.0, 101.0, 103.0, 98.0, 381.0, 82.0]
       1.0      [70.0, 343.0, 416.0, 235.0, 80.0, 207.0, 302.0...
       2.0      [337.0, 18.0, 261.0, 399.0, 36.0, 431.0, 124.0.

In [50]:
df.groupby(["block"]).goal_img.apply(lambda x: len(np.unique(x)))

block
1.0    24
2.0     1
3.0     1
4.0    24
5.0    24
6.0     1
Name: goal_img, dtype: int64

In [51]:
df.groupby(["block"]).goal_img.apply(lambda x: np.unique(x, return_counts=True)[1])

block
1.0    [2, 2, 2, 2, 2, 2, 2, 2, 6, 2, 6, 2, 2, 6, 2, ...
2.0                                                 [72]
3.0                                                 [72]
4.0    [2, 2, 2, 2, 2, 2, 6, 2, 6, 6, 2, 2, 6, 2, 6, ...
5.0    [2, 6, 2, 2, 2, 2, 2, 6, 2, 6, 2, 2, 2, 2, 2, ...
6.0                                                 [72]
Name: goal_img, dtype: object

In [53]:
df.groupby(["block", "stim"]).goal_img.apply(lambda x: np.sort(np.unique(x, return_counts=True)[1]))

block  stim
1.0    1.0                 [1, 1, 1, 3, 3, 3]
       2.0     [1, 1, 1, 1, 1, 1, 1, 1, 1, 3]
       3.0           [1, 1, 1, 1, 1, 1, 3, 3]
       4.0                 [1, 1, 1, 3, 3, 3]
       5.0     [1, 1, 1, 1, 1, 1, 1, 1, 1, 3]
       6.0           [1, 1, 1, 1, 1, 1, 3, 3]
2.0    1.0                               [12]
       2.0                               [12]
       3.0                               [12]
       4.0                               [12]
       5.0                               [12]
       6.0                               [12]
3.0    1.0                               [12]
       2.0                               [12]
       3.0                               [12]
       4.0                               [12]
       5.0                               [12]
       6.0                               [12]
4.0    1.0     [1, 1, 1, 1, 1, 1, 1, 1, 1, 3]
       2.0           [1, 1, 1, 1, 1, 1, 3, 3]
       3.0                 [1, 1, 1, 3, 3, 3]
       4.0     [1, 1, 

In [55]:
df.groupby(["block", "group"]).goal_img.apply(lambda x: np.sort(np.unique(x, return_counts=True)[1]))

block  group
1.0    0.0                  [2, 2, 2, 6, 6, 6]
       1.0      [2, 2, 2, 2, 2, 2, 2, 2, 2, 6]
       2.0            [2, 2, 2, 2, 2, 2, 6, 6]
2.0    0.0                                [24]
       1.0                                [24]
       2.0                                [24]
3.0    0.0                                [24]
       1.0                                [24]
       2.0                                [24]
4.0    0.0                  [2, 2, 2, 6, 6, 6]
       1.0      [2, 2, 2, 2, 2, 2, 2, 2, 2, 6]
       2.0            [2, 2, 2, 2, 2, 2, 6, 6]
5.0    0.0                  [2, 2, 2, 6, 6, 6]
       1.0      [2, 2, 2, 2, 2, 2, 2, 2, 2, 6]
       2.0            [2, 2, 2, 2, 2, 2, 6, 6]
6.0    0.0                                [24]
       1.0                                [24]
       2.0                                [24]
Name: goal_img, dtype: object

In [56]:
df.groupby(["block", "group"]).goal_img.apply(lambda x: len(np.unique(x)))

block  group
1.0    0.0       6
       1.0      10
       2.0       8
2.0    0.0       1
       1.0       1
       2.0       1
3.0    0.0       1
       1.0       1
       2.0       1
4.0    0.0       6
       1.0      10
       2.0       8
5.0    0.0       6
       1.0      10
       2.0       8
6.0    0.0       1
       1.0       1
       2.0       1
Name: goal_img, dtype: int64

In [26]:
df.groupby(["block", "goal_img"]).stim.apply(lambda x: len(np.unique(x)))

block  goal_img
2.0    2.0         2
       4.0         2
       65.0        2
       98.0        2
       135.0       2
       148.0       2
       183.0       2
       194.0       2
       195.0       2
       210.0       2
       221.0       2
       230.0       2
       232.0       2
       239.0       2
       288.0       2
       292.0       2
       301.0       2
       375.0       2
       377.0       2
       416.0       2
4.0    60.0        2
       100.0       2
       125.0       2
       177.0       2
       178.0       2
       188.0       2
       192.0       2
       233.0       2
       234.0       2
       265.0       2
       266.0       2
       275.0       2
       326.0       2
       339.0       2
       359.0       2
       367.0       2
       373.0       2
       396.0       2
       421.0       2
       429.0       2
5.0    30.0        2
       38.0        2
       48.0        2
       70.0        2
       81.0        2
       91.0        2
       105.0      

In [19]:
# df.to_csv("test_seq.csv")

## Quit MATLAB if needed

In [15]:
# quit matlab
if use_matlab:
    m.exit()